## Generating Music using LTSM Neural Network

Original Tutorial Found:
https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

Midi Files:
https://www.midiworld.com

In [23]:
# Importing Packages
import music21
import tensorflow.keras
import numpy
import pandas
from glob import glob
import glob2
import os
import glob
import music21
from music21 import beam
from music21 import clef
from music21 import common
from music21 import chord
from music21 import defaults
from music21 import environment
from music21 import key
from music21 import meter
from music21 import note
from music21 import pitch
from music21 import converter, instrument, note, chord
from music21.common.numberTools import opFrac
import pickle
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from music21.exceptions21 import StreamException

In [24]:
# Updating glob to use glob 2
glob = glob2.glob('/midi_songs/*')

## Training

In [25]:
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()
    # get amount of pitch names
    n_vocab = len(set(notes))
    network_input, network_output = prepare_sequences(notes, n_vocab)
    model = create_network(network_input, n_vocab)
    train(model, network_input, network_output)

In [26]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)
        print("Parsing %s" % file)
        notes_to_parse = None
        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
# Writes the notes file based on the midi files in the midi_songs folder
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [27]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 50
    # get all pitch names
    pitchnames = sorted(set(item for item in notes))
     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    network_input = []
    network_output = []
    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    n_patterns = len(network_input)
    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)
    network_output = np_utils.to_categorical(network_output)
    return (network_input, network_output)

In [28]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    print(network_input.shape[1])
    print(network_input.shape[2])

    return model

In [29]:
def train(model, network_input, network_output):
    """ train the neural network """
    # Saves each epoch generated model to the data folder
    filepath = "data/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]
# Number of epochs = 100
    model.fit(network_input, network_output, epochs=100, batch_size=128, callbacks=callbacks_list)

In [30]:
train_network()

Parsing midi_songs\Disney_Themes_-_Bippity_Boppity_Boo.mid
Parsing midi_songs\Disney_Themes_-_DuckTales.mid
Parsing midi_songs\Disney_Themes_-_Hakuna_Matata.mid
Parsing midi_songs\Disney_Themes_-_It's_A_Small_World.mid
Parsing midi_songs\Disney_Themes_-_Kiss_The_Girl.mid
Parsing midi_songs\Disney_Themes_-_Main_Street_Electrical_Parade.mid
Parsing midi_songs\Disney_Themes_-_Pirates_Of_The_Caribbean.mid
Parsing midi_songs\Disney_Themes_-_Tiki_Room.mid
Parsing midi_songs\Disney_Themes_-_Under_The_Sea.mid
Parsing midi_songs\Disney_Themes_-_Whole_New_World.mid
Parsing midi_songs\Disney_Themes_-_Zip_A_Dee_Doo_Dah.mid
50
1
Epoch 1/100
36/36 [==============================] - 125s 3s/step - loss: 5.0418